In [9]:
!pip install -q google-generativeai gradio

In [11]:
import google.generativeai as genai
import threading

genai.configure(api_key="YOUR_KEY")

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=genai.types.GenerationConfig(
        max_output_tokens=512,
        temperature=0.7,
    )
)

chat = model.start_chat(history=[])

In [13]:
def chat_with_timeout(user_input, timeout_seconds=10):
    result = {"text": "⚠️ Gemini did not respond in time."}

    def call_model():
        try:
            response = chat.send_message(user_input)
            result["text"] = response.text
        except Exception as e:
            result["text"] = f"❌ Error: {str(e)}"

    thread = threading.Thread(target=call_model)
    thread.start()
    thread.join(timeout_seconds)

    if thread.is_alive():
        return "⚠️ Gemini is taking too long. Try again."
    return result["text"]

In [15]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("Gemini 1.5 Flash Chatbot")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Ask anything...")
    clear = gr.Button("Clear Chat")

    def respond(user_msg, history):
        reply = chat_with_timeout(user_msg)
        history.append([user_msg, reply])
        return history, ""

    def reset_chat():
        global chat
        chat = model.start_chat(history=[])
        return [], ""

    msg.submit(respond, [msg, chatbot], [chatbot, msg])
    clear.click(reset_chat, None, [chatbot, msg])

demo.launch()

<ipython-input-15-552973330>:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1a1ba308fa55ed3200.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
